In [18]:
import numpy as np
import pandas as pd
import librosa as lib
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
def read_rttm(file_path):
    columns = ['Type', 'File ID', 'Channel', 'Start Time', 'Duration', 'Ortho', 'Ortho1', 'SType', 'Conf']
    df = pd.read_csv(file_path, delim_whitespace=True, header=None, names=columns)
    df['Start Time'] = df['Start Time'].astype(float)
    df['Duration'] = df['Duration'].astype(float)
    df['Conf'] = df['Conf'].astype(float)
    return df

def read_rttm_diar(file_path):
    columns = ['Type', 'File ID', 'Channel', 'Start Time', 'Duration', 'Ortho1', 'Ortho2', 'SType', 'Name1', 'Name2']
    df = pd.read_csv(file_path, delim_whitespace=True, header=None, names=columns)
    df['Start Time'] = df['Start Time'].astype(float)
    df['Duration'] = df['Duration'].astype(float)
    return df

# Rutas de los archivos (puedes parametrizar estos valores)
file_path_rttm = 'amanday-a1-nsb/amanday-a1-nsb.rttm'
file_path_diar = 'amanday-a1-nsb-diarization.rttm'

# Obtener el nombre base del archivo sin extensión ni ruta
base_filename = os.path.basename(file_path_rttm).split('.')[0]

# Crear un directorio para almacenar los resultados
output_dir = f'resultados_{base_filename}'
os.makedirs(output_dir, exist_ok=True)

# Lectura de los archivos RTTM
df_rttm = read_rttm(file_path_rttm)
df_rttm_diar = read_rttm_diar(file_path_diar)

# Modificar columnas
df_rttm['Type'] = 'Elan'
df_rttm['STypeNew'] = df_rttm['SType'].map({
    'CHI': 'KCHI',
    'MOT': 'FEM',
    'FAT': 'MAL',
    'MUJ': 'FEM',
    'NIN': 'OCH'
})
df_rttm = df_rttm[['Type', 'File ID', 'Start Time', 'Duration', 'STypeNew']]

df_rttm_diar['Type'] = 'Diar'
df_rttm_diar['STypeNew'] = df_rttm_diar['SType']
df_rttm_diar = df_rttm_diar[['Type', 'File ID', 'Start Time', 'Duration', 'STypeNew']]

# Concatenar DataFrames
df_both = pd.concat([df_rttm, df_rttm_diar])

df_both


,Type,File ID,Start Time,Duration,STypeNew
0,Elan,amanday-a1-nsb,0.000,32540.270,KCHI
1,Elan,amanday-a1-nsb,32554.300,3.750,KCHI
2,Elan,amanday-a1-nsb,32581.030,0.590,KCHI
3,Elan,amanday-a1-nsb,32599.550,0.990,KCHI
4,Elan,amanday-a1-nsb,32656.600,1.020,KCHI
...,...,...,...,...,...
8469,Diar,amanday-a1-nsbLENA1-9-3.diarization,13542.282,0.635,KCHI
8470,Diar,amanday-a1-nsbLENA1-9-3.diarization,13544.172,1.606,FEM
8471,Diar,amanday-a1-nsbLENA1-9-3.diarization,13544.179,1.581,SPEECH
8472,Diar,amanday-a1-nsbLENA1-9-3.diarization,13546.246,5.240,FEM


In [20]:
# Define las variables para el análisis
start = 3600
end = 10800
step = 0.1

# Inicializa listas para resultados
results = {
    'Elan_KCHI': [],
    'Elan_OCH': [],
    'Elan_FEM': [],
    'Elan_MAL': [],
    'Diar_KCHI': [],
    'Diar_OCH': [],
    'Diar_FEM': [],
    'Diar_MAL': []
}

# Crear la barra de progreso
num_steps = int((end - start) / step)
with tqdm(total=num_steps, desc="Processing intervals") as pbar:
    # Analizar intervalos
    time_points = np.arange(start, end, step)
    for i in time_points:
        # Verificar presencia de cada tipo en el intervalo
        for key in results.keys():
            results[key].append(int(df_both[
                (df_both['Start Time'] <= i) & 
                (df_both['Start Time'] + df_both['Duration'] > i) & 
                (df_both['STypeNew'] == key.split('_')[1]) & 
                (df_both['Type'] == key.split('_')[0])
            ].shape[0] > 0))
        pbar.update(1)

# Crear DataFrame de resultados
df_results = pd.DataFrame(results)
df_results['Time'] = time_points  # Agregar la columna de tiempo



Processing intervals: 100%|██████████| 72000/72000 [24:25<00:00, 49.13it/s]


In [22]:
df_results.to_csv("df_results_amanday-a1-nsb.csv")

In [23]:
# Cálculo de precisión, recall y F1-score
types = ['KCHI', 'OCH', 'FEM', 'MAL']

# Inicializar listas para almacenar las métricas
metrics = {
    'Archivo': [],
    'Tipo': [],
    'Precisión': [],
    'Recall': [],
    'F1-score': []
}

print("\nCalculando métricas de precisión, recall y F1-score:")
for t in tqdm(types, desc="Procesando tipos"):
    y_true = df_results['Elan_' + t]
    y_pred = df_results['Diar_' + t]
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    
    # Almacenar las métricas
    metrics['Archivo'].append(base_filename)
    metrics['Tipo'].append(t)
    metrics['Precisión'].append(precision)
    metrics['Recall'].append(recall)
    metrics['F1-score'].append(f1)
    
    # Imprimir los resultados
    print(f"\nResultados para el archivo {base_filename}, tipo {t}:")
    print(f"Precisión: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")

# Crear DataFrame de métricas
df_metrics = pd.DataFrame(metrics)

# Guardar las métricas en un archivo CSV
metrics_csv_path = os.path.join(output_dir, f'metricas_{base_filename}.csv')
df_metrics.to_csv(metrics_csv_path, index=False)
print(f"\nMétricas guardadas en: {metrics_csv_path}")

# Generar gráficos de métricas
sns.set(style="whitegrid")

# Gráfico combinado de métricas
df_metrics_melted = pd.melt(df_metrics, id_vars=['Archivo', 'Tipo'], value_vars=['Precisión', 'Recall', 'F1-score'], var_name='Métrica', value_name='Valor')

plt.figure(figsize=(10, 6))
sns.barplot(x='Tipo', y='Valor', hue='Métrica', data=df_metrics_melted)
plt.title(f'Métricas por Tipo - {base_filename}')
plt.ylim(0, 1)
plt.ylabel('Valor')
plt.legend(title='Métrica')
plt.tight_layout()

# Guardar el gráfico de métricas
metrics_plot_path = os.path.join(output_dir, f'graficos_metricas_{base_filename}.png')
plt.savefig(metrics_plot_path)
print(f"Gráfico de métricas guardado en: {metrics_plot_path}")
plt.close()


Calculando métricas de precisión, recall y F1-score:


Procesando tipos:  50%|█████     | 2/4 [00:00<00:00, 16.33it/s]


Resultados para el archivo amanday-a1-nsb, tipo KCHI:
Precisión: 1.0000
Recall: 0.1387
F1-score: 0.2435

Resultados para el archivo amanday-a1-nsb, tipo OCH:
Precisión: 0.0000
Recall: 0.0000
F1-score: 0.0000

Resultados para el archivo amanday-a1-nsb, tipo FEM:
Precisión: 0.0000
Recall: 0.0000
F1-score: 0.0000


Procesando tipos: 100%|██████████| 4/4 [00:00<00:00, 14.32it/s]



Resultados para el archivo amanday-a1-nsb, tipo MAL:
Precisión: 0.0000
Recall: 0.0000
F1-score: 0.0000

Métricas guardadas en: resultados_amanday-a1-nsb\metricas_amanday-a1-nsb.csv
Gráfico de métricas guardado en: resultados_amanday-a1-nsb\graficos_metricas_amanday-a1-nsb.png


In [24]:
# === Comienza el código para generar la gráfica específica ===

# Seleccionar las columnas que empiezan con "Diar_"
diar_columns = [col for col in df_results.columns if col.startswith('Diar_')]

# Cambiar los valores de 1 a 0.5 en esas columnas
df_results[diar_columns] = df_results[diar_columns].replace(1, 0.5)

# Definir las columnas que deseas graficar
columns_to_plot = ['Elan_FEM', 'Diar_FEM']

# Definir colores contrastantes
colors = ['#FF6347', '#4682B4']  # Tomato y SteelBlue

# Crear la gráfica de dispersión para mostrar solo los estados sin transiciones
plt.figure(figsize=(14, 6))

# Graficar solo los puntos donde el valor es 1 o 0.5
for i, column in enumerate(columns_to_plot):
    # Filtrar los puntos donde hay actividad (valor > 0)
    df_active = df_results[df_results[column] > 0]
    plt.scatter(df_active['Time'], df_active[column], label=column, color=colors[i], s=10)

# Configuración del gráfico
plt.title(f'Estados de Elan_FEM y Diar_FEM a lo largo del tiempo - {base_filename}')
plt.xlabel('Tiempo [seg]')
plt.ylabel('Valores Binarios')
plt.legend(loc='upper right')
plt.grid(True)
plt.xlim(start, end)
plt.ylim(-0.1, 1.1)  # Ajustar el límite del eje y para visualizar correctamente
plt.yticks([0, 0.5, 1], ['0', '0.5 (Diar_FEM)', '1 (Elan_FEM)'])
plt.tight_layout()

# Guardar el gráfico de estados
states_plot_path = os.path.join(output_dir, f'estados_FEM_{base_filename}.png')
plt.savefig(states_plot_path)
print(f"Gráfico de estados guardado en: {states_plot_path}")
plt.close()


Gráfico de estados guardado en: resultados_amanday-a1-nsb\estados_FEM_amanday-a1-nsb.png
